In [ ]:
import torch
import scipy
import numpy as np
from dict_layer import Model, dictloss
import os
import NES
import scipy.io as io
from utlis import survey_deisgn, slownessMap, patchSamp, getPatches, rmse, vd
from LsTomo import conventional_tomo2
import matplotlib.pyplot as plt
from scipy.sparse.linalg import lsqr
from itkm import itkm
from omp_n import omp_n
from scipy.sparse import csc_matrix
from mpl_toolkits.axes_grid1 import make_axes_locatable
####################################################################################
np.random.seed(123)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True
####################################################################################
cuda = torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)
print(torch.cuda.device_count(),torch.cuda.get_device_properties(0), cuda, device)
####################################################################################
model = Model()
model = model.to(device)
# model = torch.nn.DataParallel(model, device_ids=device_ids)
####################################################################################
save_path = './Marmousi_smooth3_noise2/'
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [ ]:
Vel2D = NES.misc.Marmousi(smooth=3, section=[[600, 900], None]) # importing from NES package data
vmin, vmax = Vel2D.min, Vel2D.max
xmin, zmin = Vel2D.xmin
xmax, zmax = Vel2D.xmax
nx, nz = 100, 100
x = np.linspace(xmin, xmax, nx)
z = np.linspace(zmin, zmax, nz)
Xr_2d = np.stack(np.meshgrid(x, z, indexing='ij'), axis=-1)
V_2d = Vel2D(Xr_2d)
sTrue = np.transpose(1/V_2d)
plt.imshow(sTrue)
plt.show()

In [ ]:
Tarr, A, vb, vb2 = survey_deisgn(64, sTrue, save_path=save_path)
noiseFrac = 0.02                                          # noise STD as fraction of mean value of travel time (0=noise free case)                                                
stdNoise = np.mean(Tarr) * noiseFrac
noise    = stdNoise * np.random.randn(Tarr.shape[0], Tarr.shape[1])
Tarr_n   = Tarr + noise                                  
Asum = np.sum(A, axis=1, keepdims=True)                                    # estimating referense slowness from travel time observations
invAsum = np.linalg.pinv(Asum)
sRef = invAsum @ Tarr_n                                     # Tarr_n: 1x2016 A: 2016x10000 vb:10000x1
#######################################################################################

w1, w2 = sTrue.shape
patch_size = 10
patches  = getPatches(w1, w2, patch_size)        # calculating image patch indices
percZero = patchSamp(A, patches)       # percentage ray coverage in patches
ss = np.zeros((vb.size, 1))
npp, npatches = patches.shape
nrays, npix   = A.shape
normError = [np.linalg.norm(Tarr_n - A @ (ss + sRef))]
print('Trave time initinal error: {}'.format(normError))
############################################################
epoch = 50
natom = 150
iters = 50
lam2  = 0
D_sparse = 1
C_sparse = 1

In [ ]:
for i in range(iters):
    dt  = Tarr_n - A @ (ss + sRef)
    spA = csc_matrix(A)
    ds  = lsqr(spA, dt, damp=10, iter_lim=1e3)[0]
    ds  = np.expand_dims(ds, axis=1)
    sg = ds + ss
    Y   = sg.flatten()[patches]
    meanY = np.mean(Y, axis=0, keepdims=True)
    Yc = Y - meanY
    Yl = Yc[:, percZero <= 0.1]
    D = itkm(Yl, natom, 2, 50)
    X = omp_n(D, Yc, 2)
    ss_b = D @ X + meanY
    ss_p_sum = np.zeros(ss.shape)
    
    for k in range(npatches):
        ss_p_sum[patches[:, k], 0] = ss_p_sum[patches[:, k], 0] + ss_b[:, k]
    
    ss_f = (lam2 * sg + ss_p_sum)/(lam2 + npp)
    ss   = ss_f * vb2
    Tref = A @ (ss + sRef)
    normError_new = np.linalg.norm(Tref - Tarr_n)
    print('Iter: {}, inversion norm error: {}'.format(i+1, normError_new))